In [1]:
import os
import pandas as pd
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

import tqdm
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split

%pip install torchsummary
%pip install efficientnet_pytorch

from torchsummary import summary
from efficientnet_pytorch import EfficientNet

print('Done')

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Done


In [2]:
test_dir = '/opt/ml/input/data/eval'
train_dir = '/opt/ml/input/data/train'

In [3]:
train_data = pd.read_csv(train_dir + '/train_labelled.csv')
train_data = train_data.iloc[:, [1, 2]]
train_data

,path,label
0,/opt/ml/input/data/train/images/000001_female_...,10
1,/opt/ml/input/data/train/images/000001_female_...,4
2,/opt/ml/input/data/train/images/000001_female_...,4
3,/opt/ml/input/data/train/images/000001_female_...,4
4,/opt/ml/input/data/train/images/000001_female_...,4
...,...,...
18895,/opt/ml/input/data/train/images/006959_male_As...,0
18896,/opt/ml/input/data/train/images/006959_male_As...,0
18897,/opt/ml/input/data/train/images/006959_male_As...,0
18898,/opt/ml/input/data/train/images/006959_male_As...,0


In [4]:
class TrainDataset(Dataset):
  def __init__(self, img_paths, transform):
    self.X = img_paths['path']
    self.y = img_paths['label']
    self.transform = transform

  def __getitem__(self, index):
    image = Image.open(self.X.iloc[index])
    label = self.y.iloc[index]

    if self.transform:
      image = self.transform(image)
    return image, torch.tensor(label)

  def __len__(self):
    return len(self.X)


class TestDataset(Dataset):
  def __init__(self, img_paths, transform):
    self.img_paths = img_paths
    self.transform = transform

  def __getitem__(self, index):
    image = Image.open(self.img_paths[index])

    if self.transform:
      image = self.transform(image)
    return image

  def __len__(self):
    return len(self.img_paths)

In [5]:
transform = torchvision.transforms.Compose([
  Resize((512, 384), Image.BILINEAR),
  ToTensor(),
  Normalize(mean=(.5, .5, .5), std=(.2, .2, .2)),
])

In [6]:
train, valid = train_test_split(train_data,
                                test_size=0.2,
                                shuffle=True,
                                stratify=train_data['label'],
                                random_state=1234)
train.shape, valid.shape

((15120, 2), (3780, 2))

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [16]:
BATCH_SIZE = 32
EPOCHS = 5

train_dataset = TrainDataset(train, transform)
train_dataloader = DataLoader(train_dataset,
                              batch_size=BATCH_SIZE,
                              shuffle=True)

valid_dataset = TrainDataset(valid, transform)
valid_dataloader = DataLoader(valid_dataset,
                              batch_size=BATCH_SIZE,
                              shuffle=True) 

In [17]:
model = EfficientNet.from_pretrained('efficientnet-b3', num_classes=18)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b3-5fb5a3c3.pth" to /opt/ml/.cache/torch/hub/checkpoints/efficientnet-b3-5fb5a3c3.pth



Loaded pretrained weights for efficientnet-b3


In [18]:
model.to(device)

loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

dataloaders = {
  'train' : train_dataloader,
  'test' : valid_dataloader
}

In [19]:
### 학습 코드 시작
best_test_accuracy = 0.
best_test_loss = 9999.

for epoch in range(EPOCHS):
  for phase in ["train", "test"]:
    running_loss = 0.
    running_acc = 0.
    if phase == "train":
      model.train() # 네트워크 모델을 train 모드로 두어 gradient을 계산하고, 여러 sub module (배치 정규화, 드롭아웃 등)이 train mode로 작동할 수 있도록 함
    elif phase == "test":
      model.eval() # 네트워크 모델을 eval 모드 두어 여러 sub module들이 eval mode로 작동할 수 있게 함

    for ind, (images, labels) in enumerate(tqdm(dataloaders[phase])):
      # (참고.해보기) 현재 tqdm으로 출력되는 것이 단순히 진행 상황 뿐인데 현재 epoch, running_loss와 running_acc을 출력하려면 어떻게 할 수 있는지 tqdm 문서를 보고 해봅시다!
      # hint - with, pbar
      images = images.to(device)
      labels = labels.to(device)

      optimizer.zero_grad() # parameter gradient를 업데이트 전 초기화함

      with torch.set_grad_enabled(phase == "train"): # train 모드일 시에는 gradient를 계산하고, 아닐 때는 gradient를 계산하지 않아 연산량 최소화
        logits = model(images)
        _, preds = torch.max(logits, 1) # 모델에서 linear 값으로 나오는 예측 값 ([0.9,1.2, 3.2,0.1,-0.1,...])을 최대 output index를 찾아 예측 레이블([2])로 변경함  
        loss = loss_fn(logits, labels)

        if phase == "train":
          loss.backward() # 모델의 예측 값과 실제 값의 CrossEntropy 차이를 통해 gradient 계산
          optimizer.step() # 계산된 gradient를 가지고 모델 업데이트

      running_loss += loss.item() * images.size(0) # 한 Batch에서의 loss 값 저장
      running_acc += torch.sum(preds == labels.data) # 한 Batch에서의 Accuracy 값 저장

    # 한 epoch이 모두 종료되었을 때,
    epoch_loss = running_loss / len(dataloaders[phase].dataset)
    epoch_acc = running_acc / len(dataloaders[phase].dataset)

    print(f"현재 epoch-{epoch}의 {phase}-데이터 셋에서 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}")
    if phase == "test" and best_test_accuracy < epoch_acc: # phase가 test일 때, best accuracy 계산
      best_test_accuracy = epoch_acc
    if phase == "test" and best_test_loss > epoch_loss: # phase가 test일 때, best loss 계산
      best_test_loss = epoch_loss
print("학습 종료!")
print(f"최고 accuracy : {best_test_accuracy}, 최고 낮은 loss : {best_test_loss}")

RuntimeError: CUDA out of memory. Tried to allocate 240.00 MiB (GPU 0; 31.75 GiB total capacity; 30.38 GiB already allocated; 193.50 MiB free; 30.43 GiB reserved in total by PyTorch)